## 문제 설명

Fasion-MNIST 데이터셋은 10개의 의류 카테고리로 구성된 흑백 이미지 데이터셋입니다. 본 예선 문제에서는 아래 두 클래스를 구분하는 **이진 분류기**를 양자 알고리즘을 활용하여 설계하는 것이 목표입니다. 아래의 이미지에서 T-shirt/top과 Shirt 라벨에 대해서만 분류하는 것입니다. 만들어야 할 모델은 양자 회로(quantum circuit)를 기반으로 하며, 잡음 없는 양자 시뮬레이터에서 실행가능해야 합니다.
- 클래스 0 : **T-shirt/top**
- 클래스 6 : **Shirt**

## 제약 조건
본 예선에서는 양자 모델 설계에 대한 현실적인 자원 제약을 고려하며, 다음과 같은 기술적 제한을 준수하여야 합니다.
- 양자 개발 프레임워크는 PennyLane을 사용합니다.
- 참가자는 최대 8 큐빗까지 사용 가능합니다.
- 사용되는 큐빗 수는 모델 전반에 걸쳐 유지되어야 하며, ancilla qubit 또는 mid-circuit measurement는 허용되지 않습니다.
- 전체 회로의 깊이(depth)는 최대 30으로 제한합니다.
- 모델 내 학습 가능한 파라미터(num_trainable_params)의 총 개수는 60개 이하로 제한합니다.
- 입력 데이터를 양자 상태로 인코딩하는 과정은 Amplitude Encoding, Angle Encoding, IQP-style Embedding 등을 자유롭게 활용할 수 있습니다.
- 참가자의 판단에 따라 데이터 차원 축소가 필요한 경우, 비지도 차원 축소 기법(PCA 등)의 사용을 허가합니다.
- 고전적 머신러닝 모델과의 하이브리드 구성은 불허하며, 양자 회로만 사용합니다.